In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
# from IPython.display import display, HTML

from pykat import finesse
from pykat.commands import *
pykat.init_pykat_plotting(dpi=90)

from pprint import pprint as pprint

In [ ]:
SMALL_SIZE = 18
MEDIUM_SIZE = 20
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
# !!! these functions use the WRONG formula for dB, should be 20*log10(-)

def r_to_gain_in_dB(r):
    return 10*np.log10(np.exp(r))

def gain_in_dB_to_r(gain):
    return np.log(10**(gain/10))

In [ ]:
# base model that holds the components, run detection models on top of this
basekat = finesse.kat()
basekat.verbose = False

basecode = """
# no spaces in model, except inside cavity

l laser 1 0 0 nbs1 # power = 1W
bs BS 0.5 0.5 0 45 nbs1 nbs2 nbs3 nbs4 # phi = 0

s sdbs1 0 nbs4 ndbs1 # zero length space
dbs isol ndbs1 ndbs2 ndbs3 ndbs4  # dbs doesn't req R T phi alpha
s sdbs2 0 ndbs2 nc0 # zero length space

m m1 0.9 0.1 0 nc0 nc1 # R = 0.9, T = 0.1, phi = 0
s sc1 0.5 nc1 nc2 # total cavity L = 1m
nle nle1 0.05 0 nc2 nc3 # r = 0.05, sqz angle = 0
s sc2 0.5 nc3 nc4 # total cavity L = 1m
m m2 1 0 0 nc4 nc5 # R = 1, T = 0, phi = 0

# homodyne detector and quantum noise
qhd qn 180 nbs2 nbs3
fsig noise 1 # noise at 1 Hz

# by default noise enters through every open port, lossy optic, or squeezer
# -- Quantum noise inputs --
#   |- %10-.10s - laser Node LASER NOISE
#   |- %10-.10s - m2 Node OPEN PORT
#   |- %10-.10s - BS Node OPEN PORT
#   |- %10-.10s - BS Node OPEN PORT
#   |- %10-.10s - isol Node OPEN PORT
#   |- %10-.10s - isol Node OPEN PORT
# can instead specify exactly where through "vacuum" command
# vacuum isol laser
#printnoises

"""

basekat.parse(basecode)

In [ ]:
pprint((basekat.components, basekat.detectors, basekat.commands))

In [ ]:
kat1 = deepcopy(basekat)
kat1.verbose = False
kat1code = """
xaxis nle1 r lin -1 1 10000 # to spot threshold
#yaxis abs:deg
"""

kat1.parse(kat1code)

# pprint((kat1.components, kat1.detectors, kat1.commands))
out1 = kat1.run()
print(out1.stdout)

In [ ]:
out1.plot(detectors=['qn'], title="homodyne intensity vs squeezer parameter")
# detectors=['qn']
out1.plot(filename='pykat_qhd_vs_r.pdf',
          title="homodyne intensity vs squeezer parameter",
          ylim=(0, 0.5*1e-8))

In [ ]:
# checking limits against HomI(0)
homI_zero = out1.y[np.searchsorted(out1.x, 0)][0]
print(out1.y[0], homI_zero, out1.y[-1])

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(out1.x, 20*np.log10(out1.y/homI_zero))
ax.set_ylim(0, 5)
ax.set(title="relative homodyne intensity vs squeezer parameter",
       xlabel="squeezer parameter, r",
       ylabel="abs of relative noise")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(out1.x, out1.y/homI_zero)
ax.set_ylim(0, 5)
ax.set(title="relative homodyne intensity vs squeezer parameter",
       xlabel="squeezer parameter, r / dB",
       ylabel="abs of relative noise")
fig.savefig("pykat_relative_qhd_vs_r.pdf")
plt.show()

In [ ]:
# finding threshold
print(out1.x[out1.y.argmin()], out1.x[out1.y.argmax()])
# rThresh
rT = 0.1144

In [ ]:
# doing it all with sq

kat2 = finesse.kat()
kat2.verbose = False

kat2code = """
l laser 1 0 0 nbs1 # power = 1W
bs BS 0.5 0.5 0 45 nbs1 nbs2 nbs3 nbs4 # phi = 0

s sdbs1 0 nbs4 ndbs1 # zero length space
dbs isol ndbs1 ndbs2 ndbs3 ndbs4  # dbs doesn't req R T phi alpha
s sdbs2 0 ndbs2 nc0 # zero length space

sq sqz1 0 0.1 90 nc0

# homodyne detector and quantum noise
qhd qn 180 nbs2 nbs3
fsig noise 1 # noise at 1 Hz

xaxis sqz1 r lin 0 1 10000 # to spot threshold
#yaxis abs:deg
"""
kat2.parse(kat2code)

out2 = kat2.run()

out2.plot(filename='sqz_cavity_Finesse_qhd_vs_r_for_sq.pdf',
          title="homodyne intensity vs squeezer parameter",
          ylim=(0, 0.5*1e-8))

In [ ]:
# kat3 = deepcopy(basekat)

# kat3code = """
# variable sqzr 0
# put nle1 r $sqzr

# xaxis laser f lin 0 5G 10000
# #yaxis abs:deg
# """

# kat3.parse(kat3code)

# out3 = kat3.run()

# out3.plot()

In [ ]:
# a hack: want to set r param but isn't co-operating, so hard code everything again
hackbase = finesse.kat()
hackbase.verbose = False

hackbasecode = """
l laser 1 0 0 nbs1 # power = 1W
bs BS 0.5 0.5 0 45 nbs1 nbs2 nbs3 nbs4 # phi = 0

s sdbs1 0 nbs4 ndbs1 # zero length space
dbs isol ndbs1 ndbs2 ndbs3 ndbs4  # dbs doesn't req R T phi alpha
s sdbs2 0 ndbs2 nc0 # zero length space

m m1 0.9 0.1 0 nc0 nc1 # R = 0.9, T = 0.1, phi = 0
s sc1 0.5 nc1 nc2 # total cavity L = 1m
s sc2 0.5 nc3 nc4 # total cavity L = 1m
m m2 1 0 0 nc4 nc5 # R = 1, T = 0, phi = 0

qhd qn 180 nbs2 nbs3
fsig noise 1 # noise at 1 Hz

xaxis noise f lin 0 1G 10000
"""

hackbase.parse(hackbasecode)

In [ ]:
# def hacksqz(sqz_r):
#     """given a squeezer parameter, return output of Finesse"""
#     hackkat = deepcopy(hackbase)

#     # power dB corresponding to r??
#     converted_r = 10*np.log10(np.exp(sqz_r))
    
#     hackcode = """
#     nle nle1 {0} 0 nc2 nc3 # sqz angle = 0
#     """.format(converted_r)

#     hackkat.parse(hackcode)

#     return hackkat.run()

# def rel_hacksqz(sqz_r):
#     """given a squeezer parameter, return output and relative y scale"""
#     zeroreference = hacksqz(0).y
#     out = hacksqz(sqz_r)
#     return out, out.y/zeroreference

In [ ]:
def dB_hacksqz(sqz_r_dB):
    """given a squeezer parameter in dB, return output of Finesse"""
    hackkat = deepcopy(hackbase)
    
    hackcode = """
    nle nle1 {0} 0 nc2 nc3 # sqz angle = 0
    """.format(sqz_r_dB)

    hackkat.parse(hackcode)

    return hackkat.run()

def dB_rel_hacksqz(sqz_r):
    """given a squeezer parameter, return output and relative y scale"""
    zeroreference = dB_hacksqz(0).y
    out = dB_hacksqz(sqz_r)
    return out, out.y/zeroreference


In [ ]:
def negative_test_dB_hacksqz(sqz_r_dB):
    """given a squeezer parameter in dB, return output of Finesse"""
    hackkat = deepcopy(hackbase)
    
    hackcode = """
    nle nle1 {0} 90 nc2 nc3 # sqz angle = 90
    """.format(sqz_r_dB)

    hackkat.parse(hackcode)

    return hackkat.run()

def negative_test_dB_rel_hacksqz(sqz_r):
    """given a squeezer parameter, return output and relative y scale"""
    zeroreference = negative_test_dB_hacksqz(0).y
    out = negative_test_dB_hacksqz(sqz_r)
    return out, out.y/zeroreference

In [ ]:
r23scaling = 20
r1, r2, r3, r4 = -rT, -rT/r23scaling, rT/r23scaling, rT
k1, k2, k3, k4 = dB_rel_hacksqz(r1), dB_rel_hacksqz(r2), dB_rel_hacksqz(r3), dB_rel_hacksqz(r4)

In [ ]:
nk1 = negative_test_dB_rel_hacksqz(rT)
nk2 = negative_test_dB_rel_hacksqz(rT/r23scaling)

In [ ]:
xax = k1[0].x

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(xax, k4[1], label="rT")
ax.plot(xax, k1[1], label="-rT")
# ax.legend(*map(reversed, ax.get_legend_handles_labels()))
ax.legend(title="squeezer parameter, r / dB", loc="upper left")
ax.set(title="relative homodyne intensity vs frequency offset",
       xlabel="frequency offset, $\Omega$ / Hz", ylabel="abs of relative homodyne intensity",
       ylim=(0, 2))
plt.show()

# values may not appear to diverge, however this comes down to choice of datapoints

In [ ]:
ang_freq_ax = 2*np.pi*k1[0].x

fig, ax = plt.subplots(figsize=(10,8))
ax.plot(ang_freq_ax, k4[1], label="r = rT")
ax.plot(ang_freq_ax, k3[1], label="r = rT/{}".format(r23scaling))
ax.plot(ang_freq_ax, np.ones(ang_freq_ax.shape), label="r = 0", color="lightgreen")
ax.plot(ang_freq_ax, k2[1], label="r = -rT/{}".format(r23scaling))
ax.plot(ang_freq_ax, nk2[1], label="r = rT/{}, $\phi$ = $\pi/2$".format(r23scaling),
        linestyle="--", color="orange")
ax.plot(ang_freq_ax, k1[1], label="r = -rT", color="darkgreen")
ax.plot(ang_freq_ax, nk1[1], label="r = rT, $\phi$ = $\pi/2$", linestyle="--", color="hotpink")
# ax.legend(*map(reversed, ax.get_legend_handles_labels()))
ax.legend(loc="upper left")
ax.set(xlabel="angular frequency offset, $\Omega$ / Hz", ylabel="abs of relative qhd",
       xlim=(0, 5e9), ylim=(0.987, 1.013))
fig.savefig("pykat_relative_qhd_vs_freq.pdf")
plt.show()

In [ ]:
# comparing threshold squeezer parameters in dB
print(10*np.log10(np.exp(0.026)),rT)

In [ ]:
# comparing FSR's
print(2*np.pi*299792458/2*1e-9, ang_freq_ax[1000:][np.argmax(k4[1][1000:])]*1e-9)

In [ ]:
# adapted from Vaishali's test code to check nle installed
testcode = """
l l1 1 0 0 n1
bs bs1 0.5 0.5 0 0 n1 n2 n3 n4

s s0 1 ni na
nle nle1 {} 90 na nb
s s1 1 nb n4

qhd qn 180 n2 n3
fsig noise 1

xaxis noise f lin 1 10G 10000
"""

testkat_null = finesse.kat()
testkat_null.verbose = False
testkat_null.parse(testcode.format(0))
out_null = testkat_null.run()

null_reference_value = out_null.y[0,0]

In [ ]:
def testkat_gain(gain):
    """input: gain in dB as parameter to finesse's nle
    output: actual gain
    """

    testkat = finesse.kat()
    testkat.verbose = False
    testkat.parse(testcode.format(gain))
    out = testkat.run()

    return 20*np.log10(out.y[0,0]/null_reference_value)

In [ ]:
gain_axis = np.linspace(0, 10, 20)
qhd_axis = [testkat_gain(x) for x in gain_axis]

In [ ]:
# recall that r = np.log(10**(gaindB/10))
fig, ax = plt.subplots(figsize=(4,3))
ax.plot(gain_axis, qhd_axis, '.')
ax.set(xlabel='nle input value / "dB"',
       title="testing Finesse squeezers: nle")
ax.set_ylabel("$20\, \log_{10}(\mathrm{relative\; qhd})$ / dB", fontsize=14)

plt.show()
fig.savefig("testing_Finesse_squeezers-nle.pdf")

In [ ]:
kat = pykat.finesse.kat()
kat.verbose = 0

# adapted from Vaishali's test code to check nle installed
testcode = """
l l1 1 0 0 n1
bs bs1 0.5 0.5 0 0 n1 n2 n3 n4

s s0 1 ni na
sq sqz1 0 10 90 nb
s s1 1 nb n4

qhd qn 180 n2 n3
fsig noise 1

xaxis sqz1 r lin 0 10 100
"""

kat.parse(testcode)
out = kat.run()
# out.plot()

In [ ]:
q = out['qn']
fig, ax = plt.subplots(figsize=(4,3))
ax.plot(out.x, 20*np.log10(q/q[0]), '--')
ax.set(xlabel="sq input value / dB",
       title="testing Finesse squeezers: sq")
ax.set_ylabel("$20\, \log_{10}(\mathrm{relative\; qhd})$ / dB", fontsize=14)
plt.show()
fig.savefig("testing_Finesse_squeezers-sq.pdf")